In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# Download training data
training_data = torchvision.datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

# Download test data
test_data = torchvision.datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

In [ ]:
# Create data loaders.
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
# Get cuda, mps or resort to cpu.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# Define model
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model =DNN().to(device)
print(model)

In [ ]:
#
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # weight decay is another regularization method

In [ ]:
def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute loss
        pred = model(X)
        loss = loss_func(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Test model before attack
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_func, optimizer)
    test(test_dataloader, model, loss_func)
print("Done!")

In [45]:
# FGSM attack
def FGSM_attack(model, image, label, epsilon):

    image.requires_grad = True # since we're acting on the image
    model.zero_grad()

    predicted = model(image)
    loss = loss_func(predicted, label)
    loss.backward()

    perturbed_image = image + epsilon * image.grad.sign()
    perturbed_image = torch.clamp(perturbed_image, 0, 1) # clip to ensure pixel values stay in the normalized [0,1] range
    return perturbed_image

In [ ]:
# Test the FGSM attack on samples from the MNIST test data

# set perturbation budget
epsilon = 0.1

# Load test set to pass each input to FGSM
test_dataloader_FGSM = DataLoader(test_data, batch_size=1, shuffle=False)


original_images =[]
perturbed_images =[]
true_labels =[]
predicted_labels = []

model.eval() # good practice to switch the model from training mode to testing mode

for x, y in test_dataloader_FGSM:
    x, y = x.to(device), y.to(device)

    original_images.append(x)
    true_labels.append(y.item())

    x_adv = FGSM_attack(model,x,y, epsilon)
    perturbed_images.append(x_adv)

    output = model(x_adv)
    _, y_predicted = torch.max(output, 1)
    predicted_labels.append(y_predicted.item())

print('Done running FGSM on', len(original_images), 'test samples')

In [ ]:
# Compute attack success
evasion_ctr =0
for i in range (len(true_labels)):
      if (true_labels[i]!=predicted_labels[i]):
        evasion_ctr+=1
print('Attack success rate:',(evasion_ctr/len(original_images))*100 )

In [ ]:
# Visualize sample original and perturbed mages for comparison
sample_size = 10 # maximum could be len(original_images)
for i in range (sample_size):
    # show true -->predicted
    print(true_labels[i], '-->', predicted_labels[i])
    # orignal
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(original_images[i].detach().squeeze().cpu().numpy(), cmap='gray')

    # perturbed counterpart
    plt.subplot(1, 2, 2)
    plt.title('Perturbed Image')
    plt.imshow(perturbed_images[i].detach().squeeze().cpu().numpy(), cmap='gray')
    plt.show()